In [10]:
import pandas as pd
import streamlit as st

In [14]:
df = pd.read_csv('data/result.csv')

max_count = int(df.groupby("slot").size().max())
result_dfs = []
for i in range(max_count):
    result_dfs.append(df.groupby("slot").nth(i).reset_index())

print(result_dfs[2])

cols = st.columns(max_count)

2025-06-20 13:25:15.483 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 13:25:15.484 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 13:25:15.484 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-20 13:25:15.486 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


   slot    lot_id result        date
0     6  AAA001.1   pass  2025-05-16
1     7  AAA001.1   pass  2025-05-16
